# Making predictions with the trained RL agent

In [16]:
import torch
import pandas as pd
from src.recsys_rl import LogToEpisodeConverter, OfflineEnv, DQN, OfflineDQNAgent

## Data loading

polars data

In [17]:
# load temporary data (zodra datapreprocessing class klaar is, kan deze weg)
import polars as pl
train_data = pl.read_parquet('data/events_sample.parquet').to_pandas()
train_data.head()

,timestamp,visitorid,event,itemid,transactionid,new_session_mark,session_id,session_start_time,session_length,gssid
0,2015-09-03 22:01:27.081,1532,0,303715,None,False,2,2015-09-03 22:01:27.081,0 days 00:05:00.350000,0000001532201509032201270300
1,2015-09-03 22:06:27.431,1532,0,303715,None,True,2,2015-09-03 22:01:27.081,0 days 00:05:00.350000,0000001532201509032206270300
2,2015-08-31 19:46:03.121,4248,0,281838,None,False,3,2015-08-31 19:46:03.121,0 days 00:01:43.021000,0000004248201508311946030103
3,2015-08-31 19:47:46.142,4248,0,44977,None,True,3,2015-08-31 19:46:03.121,0 days 00:01:43.021000,0000004248201508311947460103
4,2015-08-28 15:02:47.606,4531,0,1571,None,False,4,2015-08-28 15:02:47.606,0 days 00:02:01.135000,0000004531201508281502470121


pandas data

In [18]:
train_data = pd.read_csv(r'data\retailrocket\prepared\train-item-views_train_tr.txt',
                    sep='\t',)
display(train_data.head())
display(train_data.shape)

,SessionId,Time,ItemId,Date,Datestamp,TimeO,ItemSupport
0,1,1.462752e+09,9654,2016-05-09,1.462752e+09,2016-05-09 00:01:15.848000+00:00,399
1,1,1.462752e+09,33043,2016-05-09,1.462752e+09,2016-05-09 00:02:53.912000+00:00,195
2,1,1.462752e+09,32118,2016-05-09,1.462752e+09,2016-05-09 00:04:03.569000+00:00,67
3,1,1.462752e+09,12352,2016-05-09,1.462752e+09,2016-05-09 00:05:29.870000+00:00,327
4,1,1.462752e+09,35077,2016-05-09,1.462752e+09,2016-05-09 00:06:30.072000+00:00,102


(859459, 7)

In [19]:
test_data = pd.read_csv(r'data\retailrocket\prepared\train-item-views_test.txt',
                    sep='\t',)

display(test_data.head())
display(test_data.shape)

,SessionId,Time,ItemId,Date,Datestamp,TimeO,ItemSupport
0,289,1.464221e+09,125013,2016-05-26,1.464221e+09,2016-05-26 00:00:18.301000+00:00,10
1,289,1.464222e+09,64068,2016-05-26,1.464221e+09,2016-05-26 00:14:07.735000+00:00,30
2,289,1.464222e+09,133346,2016-05-26,1.464221e+09,2016-05-26 00:14:38.934000+00:00,36
3,289,1.464222e+09,438457,2016-05-26,1.464221e+09,2016-05-26 00:18:34.305000+00:00,6
4,289,1.464222e+09,198930,2016-05-26,1.464221e+09,2016-05-26 00:18:48.607000+00:00,10


(76895, 7)

## data convertion to state, action, reward, next_state, 

In [20]:
data_converter = LogToEpisodeConverter()
n_history = 3

data_converter.load_dataset(train_data)
data_converter.set_rewards({0: 5, 1: 8, 2: 10})
data_converter.create_ssar_tensor_episodes(n_history = n_history,
                                               mode='cpu_predicting')

Data loaded successfully.
Rewards set successfully.


KeyError: 'session_id'

## load the trained model

In [ ]:
# Load the trained agent
state_size = len(data_converter.tensor_episodes[0][0][0])  # Size of the state
action_size = max([t[1].item() for traj in data_converter.tensor_episodes for t in traj]) + 1  # Number of actions
agent = OfflineDQNAgent(state_size, action_size, mode='cpu_predicting')
agent.load_model('trained agents/DQN trained agent 20240530_085334 n_hist1.pth')


## make predictions

In [ ]:
# Make predictions
sample_states = [
    [9297, 281838,  44977]
    ] 
n_predictions = 5  # Number of top actions to predict

# Predict top actions
predicted_actions = agent.predict(sample_states, n_predictions)

# Output the predicted actions
print(f"Predicted top {n_predictions} actions: {predicted_actions}")

Predicted top 5 actions: [[139659, 50957, 146254, 50419, 236193]]


# Implementing the agent with session-rec framework

idee:

def fit() # traint de agent, waardoor ik niet meer los de agent hoef te laden.  
def predict_next() # voorspelt volgende item  
    output is een lijst met prediction scores voor items ids. niet de lijst met item ids zelf!
def predict_next_batch() #geoptimaliseerde predict_next()  (nice to have)

In [ ]:
predicted_scores = agent.predict_scores(states = sample_states,
                                        predict_for_item_ids = [139659, 50957, 146254, 50419, 236193, 0])
print(f'Predicted scores for top 5 actions and sixth itemid 0: {predicted_scores}')
print('Predicitn 0 has a verry low q value score')

NameError: name 'agent' is not defined

# Random agent

In [21]:
from src.recsys_dev import RandomPred
from src.evaluation import evaluate_sessions, MRR, HitRate
model = RandomPred()

model.fit(data=train_data)

~Training model!~


In [22]:
metrics = [MRR(20),
           HitRate(20)]

evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test_data,
                  train_data=train_data,
                  cut_off=20)

START evaluation of  76895  actions in  15969  sessions
    eval process:  0  of  76895  actions:  0.0  %
    eval process:  1000  of  76895  actions:  1.3004746732557384  %
    eval process:  2000  of  76895  actions:  2.600949346511477  %
    eval process:  3000  of  76895  actions:  3.901424019767215  %
    eval process:  4000  of  76895  actions:  5.201898693022954  %
    eval process:  5000  of  76895  actions:  6.502373366278692  %
    eval process:  6000  of  76895  actions:  7.80284803953443  %
    eval process:  7000  of  76895  actions:  9.103322712790169  %
    eval process:  8000  of  76895  actions:  10.403797386045907  %
    eval process:  9000  of  76895  actions:  11.704272059301644  %
    eval process:  10000  of  76895  actions:  13.004746732557384  %
    eval process:  11000  of  76895  actions:  14.305221405813121  %
    eval process:  12000  of  76895  actions:  15.60569607906886  %
    eval process:  13000  of  76895  actions:  16.9061707523246  %
    eval process

[('MRR@20: ',
  7.425086224620127e-05,
  'Bin: ;\nPrecision@20: ;',
  'Pos: ;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;\nPrecision@20: ;3.139763848011077e-05;5.41198235924048e-05;7.543865653592533e-05;0.00012247929254288295;0.0;0.000403248446983038;4.309973278165675e-05;0.00016920473773265651;5.124788602470148e-05;4.8732943469785574e-05;0.0;0.0;0.00011835720203574387;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;'),
 ('HitRate@20: ',
  0.00039392049371368546,
  'Bin: ;\nHitRate@20: ;',
  'Pos: ;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;\nHitRate@20: ;0.00025048531529839064;0.00034928396786587494;0.0004791566842357451;0.0006522093592043046;0.0;0.0008728542333430317;0.0003878975950349108;0.0005076142131979696;0.0006662225183211193;0.0008771929824561404;0.0;0.0;0.002012072434607646;0.0;0.0;0.0;